In [1]:
import cv2
import numpy as np 
import pandas as pd
import glob
import csv
import os
import pdb

In [2]:
#original_image_folder = '' #path to original image folder
gt_image_folder = '' #path to gt image folder
#original_images = glob.glob(original_image_folder + '*.jpg') + glob.glob(original_image_folder + '*.png') +  glob.glob(original_image_folder + '*.jpeg')
gt_images = glob.glob(gt_image_folder + '*.jpg') + glob.glob(gt_image_folder + '*.png') +  glob.glob(gt_image_folder + '*.jpeg') + glob.glob(gt_image_folder + '*.JPEG')
label_name = '' #set name of the label

## Create Bounding Box and Write CSV File:

In [3]:
os.makedirs('bbox',exist_ok=True)
training_path = './testWindow/original/' #Training data file path
new_dict = {'wd':[],'bbox':[]}

for j,i in enumerate(gt_images):
        img = cv2.imread(i)
        ret,thresh = cv2.threshold(cv2.cvtColor(img, cv2.COLOR_RGB2GRAY),127,255,cv2.THRESH_BINARY)
        image,contours,hier = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
        bbox_list = []
        x = None
        for c in contours:
            x,y,w,h = cv2.boundingRect(c)
            #pdb.set_trace()
            #only keep large boxes or 0 0 0 0 (negative example) boxes
            
            if w>1 and w>1 : #suppress small bounding boxes (optional)
                #bbox_list.extend([x,y,x+w-1,y+h-1])
                bbox_list.extend([x,y,w,h])
                cv2.rectangle(img,(x,y),(x+w-1,y+h-1),(0,255,0),1)
            else: 
                continue
        #new_dict.setdefault('fn', []).append(i.replace('.png','.jpeg').replace('\\','/').replace(gt_image_folder,training_path))
        if bbox_list: #remove gt files that do not contain any bboxes
            new_dict.setdefault('wd', []).append(i.replace('\\','/').replace(gt_image_folder,training_path).split('_')[0]+'.png')
            new_dict.setdefault('bbox', []).append(' '.join (str(p) for p in bbox_list))
            #draw bounding boxes on gt images
            cv2.imwrite('bbox/{}_BBox.png'.format(i.split('/')[1].split('.')[0]),img)
            #print (x,y,w,h)
            print('writing bbox/{}_BBox.png...'.format(j))
        else:
            #output 0,0,0,0 for empty image (negative training samples)
            bbox_list.extend([0,0,1,1])
            new_dict.setdefault('wd', []).append(i.replace('\\','/').replace(gt_image_folder,training_path).split('_')[0]+'.png')
            new_dict.setdefault('bbox', []).append(' '.join (str(p) for p in bbox_list))

writing bbox_MATest/0_BBox.png...
writing bbox_MATest/1_BBox.png...
writing bbox_MATest/2_BBox.png...
writing bbox_MATest/3_BBox.png...
writing bbox_MATest/4_BBox.png...
writing bbox_MATest/5_BBox.png...
writing bbox_MATest/6_BBox.png...
writing bbox_MATest/7_BBox.png...
writing bbox_MATest/8_BBox.png...
writing bbox_MATest/9_BBox.png...
writing bbox_MATest/10_BBox.png...
writing bbox_MATest/11_BBox.png...
writing bbox_MATest/12_BBox.png...
writing bbox_MATest/13_BBox.png...
writing bbox_MATest/14_BBox.png...
writing bbox_MATest/15_BBox.png...
writing bbox_MATest/16_BBox.png...
writing bbox_MATest/17_BBox.png...
writing bbox_MATest/18_BBox.png...
writing bbox_MATest/19_BBox.png...


In [4]:
df = pd.DataFrame(new_dict, columns=['wd','bbox'])
df.to_csv('window_test_MATLAB.csv', index=False) #change the name of the output .csv file

In [5]:
print(bbox_list)

[1571, 1090, 29, 110, 1408, 1037, 24, 110, 55, 1019, 38, 84, 1348, 985, 27, 106, 134, 982, 49, 84, 1093, 952, 50, 55, 981, 946, 47, 55, 506, 935, 47, 63, 397, 935, 50, 62, 1291, 930, 36, 59, 809, 930, 109, 60, 621, 930, 112, 58, 205, 927, 54, 38, 1220, 921, 49, 51, 275, 913, 52, 45, 1571, 906, 29, 113, 1410, 860, 26, 119, 59, 841, 27, 114, 1348, 830, 28, 101, 134, 813, 47, 108, 1096, 810, 45, 67, 979, 809, 52, 61, 395, 800, 48, 63, 505, 798, 44, 71, 806, 797, 113, 62, 1296, 796, 32, 48, 623, 796, 107, 63, 204, 789, 45, 46, 1222, 785, 47, 51, 274, 775, 48, 44, 1583, 710, 17, 115, 1413, 682, 24, 113, 54, 669, 30, 119, 983, 667, 47, 71, 1094, 665, 49, 69, 808, 662, 111, 64, 499, 662, 57, 69, 1347, 660, 35, 112, 392, 659, 54, 74, 620, 658, 113, 69, 1301, 656, 28, 47, 130, 656, 45, 107, 1227, 650, 46, 53, 200, 645, 45, 53, 268, 640, 55, 52, 1098, 527, 51, 70, 807, 527, 117, 69, 621, 525, 111, 71, 984, 524, 54, 76, 1418, 520, 29, 103, 1351, 510, 32, 107, 1226, 510, 47, 51, 503, 510, 49, 88, 

## End